<a href="https://colab.research.google.com/github/CaioPassos3/Metaheuristica/blob/main/MochilaComM%C3%BAltiplosModos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import random
import copy

1. Existem dois métodos para conseguir soluções do problema da mochila com múltiplas escolhas implementado, a colônia de formigas e a ligação de caminhos. Descubra qual deles tem retornado de maneira mais frequente a melhor solução, ou seja: a ligação de caminhos consegue melhorar as soluções?

In [22]:
# Implementação da Colônia de Formigas (ACO)
def aco_mochila(classes, capacidade, n_formigas=10, iteracoes=100, alpha=1, beta=2, evaporacao=0.5):
    n_classes = len(classes)
    feromonio = [[1 for _ in classe] for classe in classes]  # Feromônio inicial

    melhor_solucao = None
    melhor_valor = 0

    for _ in range(iteracoes):
        solucoes = []
        for _ in range(n_formigas):
            solucao = []
            peso_total = 0
            for i in range(n_classes):
                probabilidades = []
                for j in range(len(classes[i])):
                    item = classes[i][j]
                    if peso_total + item['peso'] <= capacidade:
                        prob = (feromonio[i][j] ** alpha) * ((1 / item['peso']) ** beta)
                        probabilidades.append((j, prob))
                if probabilidades:
                    total = sum(prob for _, prob in probabilidades)
                    prob_norm = [(j, prob / total) for j, prob in probabilidades]
                    escolhido = random.choices([j for j, _ in prob_norm], weights=[p for _, p in prob_norm])[0]
                    solucao.append(escolhido)
                    peso_total += classes[i][escolhido]['peso']
                else:
                    solucao.append(None)
            solucoes.append((solucao, peso_total))

        # Atualiza feromônio
        for i in range(n_classes):
            for j in range(len(classes[i])):
                feromonio[i][j] *= (1 - evaporacao)

        for solucao, peso in solucoes:
            if peso <= capacidade:
                valor = sum(classes[i][solucao[i]]['valor'] for i in range(n_classes) if solucao[i] is not None)
                if valor > melhor_valor:
                    melhor_valor = valor
                    melhor_solucao = solucao

    return melhor_solucao, melhor_valor

# Implementação da Ligação de Caminhos (Path Relinking)
def path_relinking(solucao1, solucao2, classes, capacidade):
    melhor_solucao = solucao1
    # Calcula o valor da solução inicial (solucao1)
    melhor_valor = sum(classes[i][solucao1[i]]['valor'] for i in range(len(solucao1)) if solucao1[i] is not None)

    # Itera sobre cada classe para tentar melhorar a solução
    for i in range(len(solucao1)):
        if solucao1[i] != solucao2[i]:  # Verifica se há diferença entre as soluções
            nova_solucao = solucao1.copy()  # Cria uma cópia da solução atual
            nova_solucao[i] = solucao2[i]  # Substitui o item na classe i pela escolha de solucao2

            # Calcula o peso total da nova solução
            peso = sum(classes[j][nova_solucao[j]]['peso'] for j in range(len(nova_solucao)) if nova_solucao[j] is not None)

            # Verifica se a nova solução é viável (peso <= capacidade)
            if peso <= capacidade:
                # Calcula o valor da nova solução
                valor = sum(classes[j][nova_solucao[j]]['valor'] for j in range(len(nova_solucao)) if nova_solucao[j] is not None)

                # Atualiza a melhor solução se o valor for maior
                if valor > melhor_valor:
                    melhor_solucao = nova_solucao
                    melhor_valor = valor

    return melhor_solucao, melhor_valor

# Exemplo de instância do problema
classes = [
    [{'valor': 10, 'peso': 5}, {'valor': 8, 'peso': 4}],  # Classe 1
    [{'valor': 15, 'peso': 8}, {'valor': 12, 'peso': 6}],  # Classe 2
    [{'valor': 7, 'peso': 3}, {'valor': 5, 'peso': 2}]     # Classe 3
]
capacidade = 12  # Aumentei a capacidade para permitir mais combinações

# Executa ACO
solucao_aco, valor_aco = aco_mochila(classes, capacidade, n_formigas=20, iteracoes=200)  # Aumentei os parâmetros do ACO
print(f"Solução ACO: {solucao_aco}, Valor: {valor_aco}")

# Gera uma solução de referência aleatória
solucao_referencia = [random.randint(0, len(classes[i]) - 1) for i in range(len(classes))]
print(f"Solução de Referência: {solucao_referencia}")

# Executa Path Relinking
solucao_pr, valor_pr = path_relinking(solucao_aco, solucao_referencia, classes, capacidade)
print(f"Solução Path Relinking: {solucao_pr}, Valor: {valor_pr}")

Solução ACO: [1, 1, 1], Valor: 25
Solução de Referência: [0, 0, 1]
Solução Path Relinking: [1, 1, 1], Valor: 25


A colônia de formigas encontrou uma solução que maximiza o valor total dos itens selecionados, sem exceder a capacidade da mochila.

O Path Relinking tentou melhorar essa solução, mas não encontrou uma combinação viável que resultasse em um valor maior que 25.

2. Elabore uma bateria de testes com diferentes valores de amplitude e preenchimento para as instâncias, bem como diferentes parâmetros para as heurísticas. Que conclusões você consegue chegar com a análise dos resultados?

In [23]:
# Implementação da Colônia de Formigas (ACO)
def aco_mochila(classes, capacidade, n_formigas=50, iteracoes=200, alpha=1, beta=5, evaporacao=0.3):
    n_classes = len(classes)
    feromonio = [[1 for _ in classe] for classe in classes]  # Feromônio inicial

    melhor_solucao = None
    melhor_valor = 0

    for _ in range(iteracoes):
        solucoes = []
        for _ in range(n_formigas):
            solucao = []
            peso_total = 0
            for i in range(n_classes):
                probabilidades = []
                for j in range(len(classes[i])):
                    item = classes[i][j]
                    if peso_total + item['peso'] <= capacidade:
                        prob = (feromonio[i][j] ** alpha) * ((1 / item['peso']) ** beta)
                        probabilidades.append((j, prob))
                if probabilidades:
                    total = sum(prob for _, prob in probabilidades)
                    prob_norm = [(j, prob / total) for j, prob in probabilidades]
                    escolhido = random.choices([j for j, _ in prob_norm], weights=[p for _, p in prob_norm])[0]
                    solucao.append(escolhido)
                    peso_total += classes[i][escolhido]['peso']
                else:
                    solucao.append(None)
            solucoes.append((solucao, peso_total))

        # Atualiza feromônio
        for i in range(n_classes):
            for j in range(len(classes[i])):
                feromonio[i][j] *= (1 - evaporacao)

        for solucao, peso in solucoes:
            if peso <= capacidade:
                valor = sum(classes[i][solucao[i]]['valor'] for i in range(n_classes) if solucao[i] is not None)
                if valor > melhor_valor:
                    melhor_valor = valor
                    melhor_solucao = solucao

    return melhor_solucao, melhor_valor

# Implementação da Ligação de Caminhos (Path Relinking)
def path_relinking(solucao1, solucao2, classes, capacidade):
    # Verifica se as soluções têm o mesmo tamanho
    if len(solucao1) != len(solucao2):
        raise ValueError("As soluções devem ter o mesmo tamanho.")

    melhor_solucao = solucao1
    # Calcula o valor da solução inicial (solucao1)
    melhor_valor = sum(classes[i][solucao1[i]]['valor'] for i in range(len(solucao1)) if solucao1[i] is not None)

    # Itera sobre cada classe para tentar melhorar a solução
    for i in range(len(solucao1)):
        if solucao1[i] != solucao2[i]:  # Verifica se há diferença entre as soluções
            nova_solucao = solucao1.copy()  # Cria uma cópia da solução atual
            nova_solucao[i] = solucao2[i]  # Substitui o item na classe i pela escolha de solucao2

            # Calcula o peso total da nova solução
            peso = sum(classes[j][nova_solucao[j]]['peso'] for j in range(len(nova_solucao)) if nova_solucao[j] is not None)

            # Verifica se a nova solução é viável (peso <= capacidade)
            if peso <= capacidade:
                # Calcula o valor da nova solução
                valor = sum(classes[j][nova_solucao[j]]['valor'] for j in range(len(nova_solucao)) if nova_solucao[j] is not None)

                # Atualiza a melhor solução se o valor for maior
                if valor > melhor_valor:
                    melhor_solucao = nova_solucao
                    melhor_valor = valor

    return melhor_solucao, melhor_valor

# Função para executar os testes
def executar_testes(instancias, parametros_aco):
    resultados = []
    for idx, instancia in enumerate(instancias):
        print(f"\n=== Instância {idx + 1} ===")
        classes = instancia["classes"]
        capacidade = instancia["capacidade"]

        for params in parametros_aco:
            print(f"\nParâmetros ACO: {params}")
            solucao_aco, valor_aco = aco_mochila(classes, capacidade, **params)
            print(f"Solução ACO: {solucao_aco}, Valor: {valor_aco}")

            # Gera soluções de referência variadas
            solucoes_referencia = [
                [random.randint(0, len(classes[i]) - 1) for i in range(len(classes))],  # Aleatória
                [0] * len(classes),  # Escolhe o primeiro item de cada classe
                [1] * len(classes),  # Escolhe o segundo item de cada classe
                [i % 2 for i in range(len(classes))]  # Alterna entre primeiro e segundo item
            ]

            for sol_ref in solucoes_referencia:
                print(f"Solução de Referência: {sol_ref}")
                try:
                    solucao_pr, valor_pr = path_relinking(solucao_aco, sol_ref, classes, capacidade)
                    print(f"Solução Path Relinking: {solucao_pr}, Valor: {valor_pr}")
                except ValueError as e:
                    print(f"Erro no Path Relinking: {e}")

                resultados.append({
                    "instancia": idx + 1,
                    "parametros_aco": params,
                    "solucao_referencia": sol_ref,
                    "solucao_aco": solucao_aco,
                    "valor_aco": valor_aco,
                    "solucao_pr": solucao_pr if 'solucao_pr' in locals() else None,
                    "valor_pr": valor_pr if 'valor_pr' in locals() else None
                })
    return resultados

# Definição das instâncias do problema
instancia_1 = {
    "classes": [
        [{'valor': 5, 'peso': 2}, {'valor': 4, 'peso': 1}],
        [{'valor': 6, 'peso': 3}, {'valor': 3, 'peso': 2}],
        [{'valor': 7, 'peso': 4}, {'valor': 2, 'peso': 1}]
    ],
    "capacidade": 10  # Aumentei a capacidade
}

instancia_2 = {
    "classes": [
        [{'valor': 10, 'peso': 5}, {'valor': 8, 'peso': 4}],
        [{'valor': 15, 'peso': 8}, {'valor': 12, 'peso': 6}],
        [{'valor': 7, 'peso': 3}, {'valor': 5, 'peso': 2}],
        [{'valor': 9, 'peso': 4}, {'valor': 6, 'peso': 3}]
    ],
    "capacidade": 15  # Aumentei a capacidade
}

instancia_3 = {
    "classes": [
        [{'valor': 20, 'peso': 10}, {'valor': 15, 'peso': 8}],
        [{'valor': 25, 'peso': 12}, {'valor': 18, 'peso': 9}],
        [{'valor': 30, 'peso': 15}, {'valor': 22, 'peso': 11}],
        [{'valor': 12, 'peso': 6}, {'valor': 10, 'peso': 5}],
        [{'valor': 14, 'peso': 7}, {'valor': 9, 'peso': 4}]
    ],
    "capacidade": 30  # Aumentei a capacidade
}

# Parâmetros para o ACO
parametros_aco = [
    {"n_formigas": 50, "iteracoes": 200, "alpha": 1, "beta": 5, "evaporacao": 0.3},
    {"n_formigas": 100, "iteracoes": 500, "alpha": 2, "beta": 3, "evaporacao": 0.2},
    {"n_formigas": 200, "iteracoes": 1000, "alpha": 1, "beta": 4, "evaporacao": 0.1}
]

# Executa os testes
resultados = executar_testes([instancia_1, instancia_2, instancia_3], parametros_aco)


=== Instância 1 ===

Parâmetros ACO: {'n_formigas': 50, 'iteracoes': 200, 'alpha': 1, 'beta': 5, 'evaporacao': 0.3}
Solução ACO: [1, 0, 0], Valor: 17
Solução de Referência: [1, 1, 1]
Solução Path Relinking: [1, 0, 0], Valor: 17
Solução de Referência: [0, 0, 0]
Solução Path Relinking: [0, 0, 0], Valor: 18
Solução de Referência: [1, 1, 1]
Solução Path Relinking: [1, 0, 0], Valor: 17
Solução de Referência: [0, 1, 0]
Solução Path Relinking: [0, 0, 0], Valor: 18

Parâmetros ACO: {'n_formigas': 100, 'iteracoes': 500, 'alpha': 2, 'beta': 3, 'evaporacao': 0.2}
Solução ACO: [0, 0, 0], Valor: 18
Solução de Referência: [1, 0, 0]
Solução Path Relinking: [0, 0, 0], Valor: 18
Solução de Referência: [0, 0, 0]
Solução Path Relinking: [0, 0, 0], Valor: 18
Solução de Referência: [1, 1, 1]
Solução Path Relinking: [0, 0, 0], Valor: 18
Solução de Referência: [0, 1, 0]
Solução Path Relinking: [0, 0, 0], Valor: 18

Parâmetros ACO: {'n_formigas': 200, 'iteracoes': 1000, 'alpha': 1, 'beta': 4, 'evaporacao': 0

A colônia de formigas está encontrando soluções boas, já o Path Relinking não está melhorando as soluções, isso pode ocorrer porque as soluções da colônia de formigas já são ótimas.
